In [23]:
#https://towardsdatascience.com/k-means-clustering-chardonnay-reviews-using-scikit-learn-nltk-9df3c59527f3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer

In [24]:
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

In [25]:
df = pd.read_csv('./woohoo1/0.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

df

,Bios,Year,Major_Dept,Ambition,Goals,Music,Sports,Friday_Nights,School_balance,Movie_Genres,Social_Style,Transfer,cluster #
0,"Cooking, foodie, baker",2,6,3,1,5,10,1,2,5,2,2,0
1,"coffee, hungry, singer 😀",3,2,1,1,7,1,2,1,8,2,1,0
2,"social-media, twitter, facebook, vape",2,1,2,1,7,6,1,1,9,1,1,0
3,"introvert, gamer, nerd",1,7,2,1,10,7,2,2,8,1,1,0
4,"Computer-science major, soccer",1,2,2,1,5,9,1,1,4,3,2,0
5,"travelling, musician artist,😀",3,4,1,1,7,10,2,3,1,3,2,0
6,"mathematics, geek, compsci",4,2,1,1,5,5,1,1,10,3,1,0
7,"Love to meet new people! I like to sing, check...",1,8,3,1,3,2,2,1,5,3,2,0
8,hello there :),3,6,3,1,2,2,1,3,10,2,1,0
9,"Travelling freak, biking is life, bball",4,5,1,1,4,9,1,3,3,2,2,0


In [26]:
# #create a list of stop words
# stop_words = set(stopwords.words("english"))
# #show how many words are in the list of stop words
# # print(len(stop_words))
# #179
# #construct a new list to store the cleaned text
# clean_desc = []

# def clean_text(text):
#     text = text.lower()
#     print(text)
#     string=''

#     text=re.sub(r"[-()\"#!@$%^&*{}?.,:]"," ", str(text))
#     text=re.sub(r"\s+"," ", str(text))
#     text=re.sub('[A-Aa-z0-9]+', ' ', str(text))

#     for word in text.split():
#         if word not in stop_words:
#             string += WordNetLemmatizer().lemmatize(word)+ ' '
#         return string

In [27]:
# df['Bios']=df['Bios'].apply(clean_text)

In [28]:
def remove_punct(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct
df['Bios'] = df['Bios'].apply(lambda x: remove_punct(x))
df.head()

,Bios,Year,Major_Dept,Ambition,Goals,Music,Sports,Friday_Nights,School_balance,Movie_Genres,Social_Style,Transfer,cluster #
0,Cooking foodie baker,2,6,3,1,5,10,1,2,5,2,2,0
1,coffee hungry singer 😀,3,2,1,1,7,1,2,1,8,2,1,0
2,socialmedia twitter facebook vape,2,1,2,1,7,6,1,1,9,1,1,0
3,introvert gamer nerd,1,7,2,1,10,7,2,2,8,1,1,0
4,Computerscience major soccer,1,2,2,1,5,9,1,1,4,3,2,0


In [29]:
tokenizer = RegexpTokenizer(r'\w+')
df['Bios'] = df['Bios'].apply(lambda x: tokenizer.tokenize(x.lower()))
df.head()

,Bios,Year,Major_Dept,Ambition,Goals,Music,Sports,Friday_Nights,School_balance,Movie_Genres,Social_Style,Transfer,cluster #
0,"[cooking, foodie, baker]",2,6,3,1,5,10,1,2,5,2,2,0
1,"[coffee, hungry, singer]",3,2,1,1,7,1,2,1,8,2,1,0
2,"[socialmedia, twitter, facebook, vape]",2,1,2,1,7,6,1,1,9,1,1,0
3,"[introvert, gamer, nerd]",1,7,2,1,10,7,2,2,8,1,1,0
4,"[computerscience, major, soccer]",1,2,2,1,5,9,1,1,4,3,2,0


In [30]:
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

In [31]:
df['Bios'] = df['Bios'].apply(lambda x: remove_stopwords(x))
df.head()

,Bios,Year,Major_Dept,Ambition,Goals,Music,Sports,Friday_Nights,School_balance,Movie_Genres,Social_Style,Transfer,cluster #
0,"[cooking, foodie, baker]",2,6,3,1,5,10,1,2,5,2,2,0
1,"[coffee, hungry, singer]",3,2,1,1,7,1,2,1,8,2,1,0
2,"[socialmedia, twitter, facebook, vape]",2,1,2,1,7,6,1,1,9,1,1,0
3,"[introvert, gamer, nerd]",1,7,2,1,10,7,2,2,8,1,1,0
4,"[computerscience, major, soccer]",1,2,2,1,5,9,1,1,4,3,2,0


In [32]:
lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text

In [33]:
df['Bios'] = df['Bios'].apply(lambda x: word_lemmatizer(x))
df.head()

,Bios,Year,Major_Dept,Ambition,Goals,Music,Sports,Friday_Nights,School_balance,Movie_Genres,Social_Style,Transfer,cluster #
0,"[cooking, foodie, baker]",2,6,3,1,5,10,1,2,5,2,2,0
1,"[coffee, hungry, singer]",3,2,1,1,7,1,2,1,8,2,1,0
2,"[socialmedia, twitter, facebook, vape]",2,1,2,1,7,6,1,1,9,1,1,0
3,"[introvert, gamer, nerd]",1,7,2,1,10,7,2,2,8,1,1,0
4,"[computerscience, major, soccer]",1,2,2,1,5,9,1,1,4,3,2,0


In [34]:
stemmer = PorterStemmer()
def word_stemmer(text):
    stem_text = " ".join([stemmer.stem(i) for i in text])
    return stem_text

df['Bios'] = df['Bios'].apply(lambda x: word_stemmer(x))
df.head()

,Bios,Year,Major_Dept,Ambition,Goals,Music,Sports,Friday_Nights,School_balance,Movie_Genres,Social_Style,Transfer,cluster #
0,cook foodi baker,2,6,3,1,5,10,1,2,5,2,2,0
1,coffe hungri singer,3,2,1,1,7,1,2,1,8,2,1,0
2,socialmedia twitter facebook vape,2,1,2,1,7,6,1,1,9,1,1,0
3,introvert gamer nerd,1,7,2,1,10,7,2,2,8,1,1,0
4,computersci major soccer,1,2,2,1,5,9,1,1,4,3,2,0


In [35]:
# #create a list of stop words
stop_words = set(stopwords.words("english"))
# #show how many words are in the list of stop words
# print(len(stop_words))
# #179
# #construct a new list to store the cleaned text
# clean_desc = []
# for w in range(len(df.Bios)):
#     desc = df['Bios'].str.lower

#     #remove punctuation
#     desc = re.sub('[^a-zA-Z]', ' ', str(desc))
    
#     #remove tags
#     desc = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",desc)
    
#     #remove special characters and digits
#     desc = re.sub("(\\d|\\W)+"," ",desc)

#     split_text = desc.split()

#     #Lemmatisation
#     lem = WordNetLemmatizer()
#     split_text = [lem.lemmatize(word) for word in split_text if not word in stop_words and len(word) >2] 

#     split_text = " ".join(split_text)
#     clean_desc.append(split_text)

In [36]:
df

,Bios,Year,Major_Dept,Ambition,Goals,Music,Sports,Friday_Nights,School_balance,Movie_Genres,Social_Style,Transfer,cluster #
0,cook foodi baker,2,6,3,1,5,10,1,2,5,2,2,0
1,coffe hungri singer,3,2,1,1,7,1,2,1,8,2,1,0
2,socialmedia twitter facebook vape,2,1,2,1,7,6,1,1,9,1,1,0
3,introvert gamer nerd,1,7,2,1,10,7,2,2,8,1,1,0
4,computersci major soccer,1,2,2,1,5,9,1,1,4,3,2,0
5,travel musician artist,3,4,1,1,7,10,2,3,1,3,2,0
6,mathemat geek compsci,4,2,1,1,5,5,1,1,10,3,1,0
7,love meet new peopl like sing checkout youtub ...,1,8,3,1,3,2,2,1,5,3,2,0
8,hello,3,6,3,1,2,2,1,3,10,2,1,0
9,travel freak bike life bball,4,5,1,1,4,9,1,3,3,2,2,0


In [37]:
#TF-IDF vectorizer
tfv = TfidfVectorizer(stop_words = "english",ngram_range = (1,1))
#transform
vec_text = tfv.fit_transform(df.Bios)
#returns a list of words.
words = tfv.get_feature_names()
print(words)
# tfidf = TfidfVectorizer(
#     min_df = 5,
#     max_df = 0.95,
#     max_features = 8000,
#     stop_words = 'english'
# )
# tfidf.fit(df['Bios'])
# text = tfidf.transform(df['Bios'])

# def get_top_keywords(data, clusters, labels, n_terms):
#     df = pd.DataFrame(data.todense()).groupby(clusters).mean()
    
#     for i,r in df.iterrows():
#         print('\nCluster {}'.format(i))
#         print(','.join([labels[t] for t in np.argsort(r)[-n_terms:]]))
            
# get_top_keywords(text, clusters, tfidf.get_feature_names(), 10)

['artist', 'baconlov', 'baker', 'basketbal', 'bball', 'bike', 'channel', 'checkout', 'coffe', 'compsci', 'computersci', 'cook', 'facebook', 'foodi', 'freak', 'friend', 'game', 'gamer', 'geek', 'gym', 'hello', 'hungri', 'internet', 'introvert', 'let', 'life', 'like', 'live', 'look', 'love', 'major', 'mathemat', 'meet', 'musician', 'nerd', 'new', 'peopl', 'player', 'problem', 'sing', 'singer', 'soccer', 'socialmedia', 'solver', 'travel', 'twitter', 'vape', 'woohoo', 'youtub']


In [38]:
#setup kmeans clustering
kmeans = KMeans(n_clusters = 5, n_init = 17, tol = 0.01, max_iter = 200)
#fit the data 
kmeans.fit(vec_text)
#this loop transforms the numbers back into words
common_words = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))
    

0 : hello, cook, coffe, introvert, foodi, hungri, nerd, baker, singer, gamer
1 : travel, artist, musician, life, bball, bike, freak, hello, gym, geek
2 : computersci, soccer, major, cook, hello, gym, geek, gamer, game, friend
3 : baconlov, basketbal, gym, live, player, youtub, facebook, hello, geek, gamer
4 : geek, compsci, mathemat, let, game, internet, woohoo, channel, checkout, hungri
